#RECS Data

In [1]:
import pickle, sys
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.distributions.mixture_rvs import mixture_rvs
import matplotlib.pyplot as plt
import numpy as np
import itertools
from matplotlib import patheffects
%matplotlib inline
recs_file = 'recs2009_public.csv'
recs_microdata = pd.read_csv(recs_file)


In [2]:
class recs_data:
    """ Object containing RECS data """
    def __init__(self, recs_microdata):
        self.fields = {'site':'TOTALBTU',
                       'electricity':'BTUEL',
                       'gas':'BTUNG',
                       'oil':'BTUFO',
                       'propane':'BTULP'}
        stories_dict =  {10:'1',
                         20:'2',
                         31:'3+',
                         32:'3+',
                         40:'2',
                         50:'3+'}
#         stories_dict =  {10:1,
#                          20:2,
#                          31:3,
#                          32:4,
#                          40:2,
#                          50:pd.np.nan}
        self.by_vintage = self._aggregate_recs(recs_microdata, parameter='YEARMADERANGE')
        self.by_fuel = self._aggregate_recs(recs_microdata, parameter='FUELHEAT')
        self.by_region = self._aggregate_recs(recs_microdata, parameter='CR')
        self.by_region_vintage = self._aggregate_recs(recs_microdata, parameters=['CR','YEARMADERANGE'], columns=vintages)
        self.by_region_fuel = self._aggregate_recs(recs_microdata, parameters=['CR','FUELHEAT'], columns=fuels)
        self.by_vintage_fuel = self._aggregate_recs(recs_microdata, parameters=['YEARMADERANGE','FUELHEAT'], columns=fuels)

        self.by_region_electric = {}
        self.by_region_nonelectric = {}
        self.by_vintage_electric = {}
        self.by_vintage_nonelectric = {}
        self.by_size_electric = {}
        self.by_size_nonelectric = {}
       
        self.by_region_fuel['weights']['Non-Electric'] = self.by_region_fuel['weights'][['Natural Gas','Propane/LPG','Fuel Oil','None']].sum(axis=1)
        for field_name, data in self.by_region_fuel.iteritems():
            if '_perhouse' in field_name:
                field_name_agg = field_name.split('_')[0]
                #print self.by_region_fuel[field_name_agg]
                #sys.exit()
                #self.by_region_nonelectric[field_name_agg]['Non-Electric'] = self.by_region_fuel[field_name_agg][['Natural Gas','Propane/LPG','Fuel Oil','None']].sum(axis=1)
                #self.by_region_nonelectric[field_name]['Non-Electric'] = (self.by_region_nonelectric[field_name_agg]['Non-Electric'] /
                #                                                          self.by_region_fuel['weights']['Non-Electric'])
        
                #self.by_region_electric[field_name_agg]['Electric'] = self.by_region_fuel[field_name_agg]['Electricity']
                #self.by_region_electric[field_name]['Electric'] = (self.by_region_electric[field_name_agg]['Electric'] /
                #                                                   self.by_region_fuel['weights']['Electricity'])                
                
        self.weights = recs_microdata['NWEIGHT']
        self.microdata = recs_microdata.loc[:,['CR','REPORTABLE_DOMAIN','YEARMADERANGE','FUELHEAT','NWEIGHT','TOTSQFT_EN','TOTALBTU','BTUEL','BTUNG','BTULP','BTUFO','STORIES',
                                               'TOTHSQFT','TOTCSQFT','TOTROOMS','TEMPHOMEAC','TEMPGONEAC','TEMPNITEAC','TEMPHOME','TEMPGONE','TEMPNITE','HDD65','CDD65','HDD30YR','CDD30YR',
                                               'CONCRETE','CRAWL','CELLAR','MONEYPY','DOLELSPH','DOLLAREL','KWH','KWHSPH','DOLLARNG','DOLNGSPH',
                                               'EQUIPAUX','REVERSE','WARMAIR','STEAMR','PERMELEC','PIPELESS','ROOMHEAT','WOODKILN','CARRYEL','CARRYKER','CHIMNEY','RANGE','DIFEQUIP','FURNFUEL','RADFUEL',
                                               'EQUIPNOHEAT','FUELNOHEAT','EQMAMT','USENG','UGWARM','UGASAUX','UGWATER','UGCOOK','UGOTH','ELWARM','ELECAUX',
                                               'SWIMPOOL','POOL','FUELPOOL','RECBATH','FUELTUB']]

        self.microdata.columns = ['CR','RD','Vintage','Heating Fuel','HouseCountScaled','Floor Area','site','electricity','gas','propane','oil','Stories',
                                  'Heated Floor Area','Cooled Floor Area','Number of Rooms','Cooling Setpoint','Cooling Setpoint Away','Cooling Setpoint Night','Heating Setpoint','Heating Setpoint Away','Heating Setpoint Night','HDD65','CDD65','HDD30YR','CDD30YR',
                                  'Slab','Crawlspace','Basement','Income','ElecExpHtg','ElecExp','kWh','kWh Htg','GasExp','GasExpHtg',
                                  'Aux Heat Used','Aux Heat Pump','Aux Furnace','Aux Boiler','Aux Built-in Elec','Aux Pipeless Furnace','Aux Room Heaters','Aux Htg Stove','Aux Portable Elec','Aux Portable Kerosene','Aux Fireplace','Aux Cooking Stove','Aux Other','Aux Furnace Fuel','Aux Boiler Fuel',
                                  'Unused Heat System Type','Unused Heat Fuel Type','Portion Main Heat','Gas Used','Gas Used Main Heat','Gas Used Aux Heat','Gas Used DHW','Gas Used Cooking','Gas Used Other','Elec Used Main Heat','Elec Used Aux Heat',
                                  'Has Swimming Pool','Has Heated Pool','Pool Heating Fuel','Has Hot Tub','Hot Tub Fuel']
        
        # Expense Calcs
        self.microdata['Elec Cost'] = self.microdata['ElecExp'] / self.microdata['kWh']
        self.microdata['Elec Cost Htg'] = self.microdata['ElecExpHtg'] / self.microdata['kWh Htg']
        self.microdata['Gas Cost'] = self.microdata['GasExp'] / (self.microdata['gas'] / 100)

        self._assign_sizes()
        for field_name, recs_field in self.fields.iteritems():
            self.microdata[field_name] = self.microdata[field_name] / 1000. # Convert kBtu to MBtu
            setattr(self, field_name, recs_microdata[recs_field] / 1000.) # Convert kBtu to MBtu
            
        for num, name in stories_dict.iteritems():
            self.microdata['Stories'].replace(num,name, inplace=True)
        
        #Multiple foundation types:
        self.microdata['Crawl+Slab'] = self.microdata['Slab'] & self.microdata['Crawlspace']
        self.microdata['Bsmt+Slab'] = self.microdata['Slab'] & self.microdata['Basement']
        self.microdata['Bsmt+Crawl'] = self.microdata['Basement'] & self.microdata['Crawlspace']
        
        for field in ['Slab','Crawlspace','Basement','Crawl+Slab','Bsmt+Crawl','Bsmt+Slab']:
            self.microdata[field] *= self.microdata['HouseCountScaled']
        
    def _assign_sizes(self):
        self.microdata.loc[:,'Size'] = pd.np.nan
        self.microdata.loc[:,'Conditioned Floor Area'] = self.microdata.loc[:,['Heated Floor Area','Cooled Floor Area']].max(axis=1)
        self.microdata.loc[(self.microdata['Conditioned Floor Area'] < 1500),'Size'] = '0-1499'
        self.microdata.loc[(self.microdata['Conditioned Floor Area'] >= 1500) & (self.microdata['Conditioned Floor Area'] < 2500),'Size'] = '1500-2499'
        self.microdata.loc[(self.microdata['Conditioned Floor Area'] >= 2500) & (self.microdata['Conditioned Floor Area'] < 3500),'Size'] = '2500-3499'
        self.microdata.loc[(self.microdata['Conditioned Floor Area'] >= 3500) & (self.microdata['Conditioned Floor Area'] < 4500),'Size'] = '3500-4499'
        self.microdata.loc[(self.microdata['Conditioned Floor Area'] >= 4500),'Size'] = '4500+'
    
    def _aggregate_recs(self, recs_microdata, parameter=None, parameters=None, columns=None):
        d = {}
        # One Parameter
        if not parameter is None:
            aggregated = recs_microdata.groupby(parameter, as_index=False).sum().set_index(parameter)
            d['weights'] = aggregated['NWEIGHT']
            
            for field_name, recs_field in self.fields.iteritems():
                d[field_name] = aggregated[recs_field + '_WEIGHTED'] / 1000. # Convert kBtu to MBtu
                d[field_name + '_perhouse'] = d[field_name] / d['weights'] #aggregated[recs_field] / 1000. # Convert kBtu to MBtu
        # Two Parameters
        elif not parameters is None:
            aggregated = recs_microdata.groupby(parameters, as_index=False).sum().reset_index()
            d['weights'] = aggregated.pivot(index=parameters[0], columns=parameters[1], values='NWEIGHT').reindex(columns=columns)
    
            for field_name, recs_field in self.fields.iteritems():
                d[field_name] = aggregated.pivot(index=parameters[0], columns=parameters[1], values=recs_field + '_WEIGHTED').reindex(columns=columns) / 1000. # Convert kBtu to MBtu
                d[field_name + '_perhouse'] = d[field_name] / d['weights'] #aggregated.pivot(index=parameters[0], columns=parameters[1], values=recs_field).reindex(columns=columns) / 1000. # Convert kBtu to MBtu

        return d

In [3]:
reloaddata = True
if not reloaddata:
    with open('recs.pickle', 'rb') as pickle_file:
        recs = pickle.load(pickle_file)
    type(exit).__repr__ = lambda s: setattr(s.shell, 'exit_now', True) or ''
    #sys.exit()

In [4]:
# Use Single-Family Detached only
recs_microdata = recs_microdata[recs_microdata['TYPEHUQ'] == 2]

In [5]:
# Convert income code to dollar values
incomes = {1:1250,
           2:3749.5,
           3:6249.5,
           4:8749.5,
           5:12499.5,
           6:17499.5,
           7:22499.5,
           8:27499.5,
           9:32499.5,
           10:37499.5,
           11:42499.5,
           12:47499.5,
           13:52499.5,
           14:57499.5,
           15:62499.5,
           16:67499.5,
           17:72499.5,
           18:77499.5,
           19:82499.5,
           20:87499.5,
           21:92499.5,
           22:97499.5,
           23:109999.5,
           24:250000}
# recs_microdata['MONEYPY'].replace(incomes, inplace=True)

# Convert Reportable Domain to Custom Region
region_def = {1:4,
              2:4,
              3:7,
              4:7,
              5:7,
              6:5,
              7:5,
              8:5,
              9:2,
              10:2,
              11:8,
              12:8,
              13:8,
              14:8,
              15:9,
              16:9,
              17:9,
              18:9,
              19:8,
              20:9,
              21:9,
              22:3,
              23:3,
              24:10,
              25:10,
              26:11,
              27:6}
cr_str = {1:'CR01',2:'CR02',3:'CR03',4:'CR04',5:'CR05',6:'CR06',7:'CR07',8:'CR08',9:'CR09',10:'CR10',11:'CR11',12:'CR12'}
recs_microdata['CR'] = recs_microdata['REPORTABLE_DOMAIN']
recs_microdata['CR'].replace(region_def, inplace=True)

# Split out Kentucky and put in 8:
recs_microdata.ix[(recs_microdata['REPORTABLE_DOMAIN'] == 18) & (recs_microdata['AIA_Zone'] == 3), 'CR'] = 8

# Split out Hawaii and put in 12:
recs_microdata.ix[(recs_microdata['REPORTABLE_DOMAIN'] == 27) & ((recs_microdata['AIA_Zone'] == 5) | (recs_microdata['HDD65'] < 4000)), 'CR'] = 12

# Split out Alaska and put in 1:
recs_microdata.ix[(recs_microdata['REPORTABLE_DOMAIN'] == 27) & (recs_microdata['HDD65'] > 6930), 'CR'] = 1 #Source for 6930 HDD: Dennis Barley

# Combine 2000–2004 and 2005–2009:
recs_microdata.ix[(recs_microdata['YEARMADERANGE'] == 8), 'YEARMADERANGE'] = 7

# Rename fields
vintages = ['pre-1950','1950s', '1960s','1970s','1980s','1990s','2000s']
vintage_dict = dict((i+1, x) for (i, x) in enumerate(vintages))
recs_microdata['YEARMADERANGE'].replace(vintage_dict, inplace=True)

fuels = ['Electricity','Fuel Oil', 'Natural Gas', 'Propane/LPG', 'None']
fuels_dict = {1:'Natural Gas', 2:'Propane/LPG', 3:'Fuel Oil', 5:'Electricity', 21:'Other', -2:'None'}

for field in ['FUELHEAT','FUELPOOL','FUELTUB']:
    # Combine Other Fuels
    recs_microdata.ix[(recs_microdata[field] == 4) | (recs_microdata[field] == 7) | (recs_microdata[field] == 8) | (recs_microdata[field] == 9), field] = 21
    # Map fuel strings
    recs_microdata[field].replace(fuels_dict, inplace=True)

# Drop homes with "Other" main heating fuel
recs_microdata = recs_microdata[recs_microdata['FUELHEAT'] != 'Other']

# Drop Alaska and Hawaii
recs_microdata = recs_microdata[recs_microdata['CR'] != 1]
recs_microdata = recs_microdata[recs_microdata['CR'] != 12]

# Replace CR strings
recs_microdata['CR'].replace(cr_str, inplace=True)

# Replace NAN in Has Heated Pool with 0
recs_microdata['POOL'].replace(-2,0, inplace=True)

In [6]:
# Correct inconsistency between FUELHEAT and UGWAMR
#print len(recs_microdata[(recs_microdata['UGWARM'] == 1) & (recs_microdata['FUELHEAT'] == 'Electricity')])
recs_microdata.ix[(recs_microdata['UGWARM'] == 1),'FUELHEAT'] = 'Natural Gas'
#print len(recs_microdata[(recs_microdata['UGWARM'] == 1) & (recs_microdata['FUELHEAT'] == 'Electricity')])

###Apply Weights

In [7]:
for col in recs_microdata.columns:
    for word in ['TOTAL', 'BTU', 'KWH']:
        if word in col:
            recs_microdata[col + '_WEIGHTED'] = recs_microdata[col] * recs_microdata['NWEIGHT']
'{:,.2f} QBtu'.format(recs_microdata['TOTALBTU_WEIGHTED'].sum() / 1e12)

'7.36 QBtu'

###Do Aggregations

In [8]:
recs = recs_data(recs_microdata)

In [9]:
with open('recs.pickle', 'wb') as pickle_file:
    pickle.dump(recs, pickle_file)

In [10]:
recs_microdata['HDD_weighted'] = recs_microdata['HDD65'] * recs_microdata['NWEIGHT']

In [11]:
g = recs_microdata.groupby('REPORTABLE_DOMAIN').sum()

In [12]:
recs.microdata.columns

Index([u'CR', u'RD', u'Vintage', u'Heating Fuel', u'HouseCountScaled', u'Floor Area', u'site', u'electricity', u'gas', u'propane', u'oil', u'Stories', u'Heated Floor Area', u'Cooled Floor Area', u'Number of Rooms', u'Cooling Setpoint', u'Cooling Setpoint Away', u'Cooling Setpoint Night', u'Heating Setpoint', u'Heating Setpoint Away', u'Heating Setpoint Night', u'HDD65', u'CDD65', u'HDD30YR', u'CDD30YR', u'Slab', u'Crawlspace', u'Basement', u'Income', u'ElecExpHtg', u'ElecExp', u'kWh', u'kWh Htg', u'GasExp', u'GasExpHtg', u'Aux Heat Used', u'Aux Heat Pump', u'Aux Furnace', u'Aux Boiler', u'Aux Built-in Elec', u'Aux Pipeless Furnace', u'Aux Room Heaters', u'Aux Htg Stove', u'Aux Portable Elec', u'Aux Portable Kerosene', u'Aux Fireplace', u'Aux Cooking Stove', u'Aux Other', u'Aux Furnace Fuel', u'Aux Boiler Fuel', u'Unused Heat System Type', u'Unused Heat Fuel Type', u'Portion Main Heat', u'Gas Used', u'Gas Used Main Heat', u'Gas Used Aux Heat', u'Gas Used DHW', u'Gas Used Cooking', u'Gas

In [13]:
# recs.microdata['gas_weighted'] = recs.microdata['gas'] * recs.microdata['HouseCountScaled']
# recs.microdata['HDD_weighted'] = recs.microdata['HDD65'] * recs.microdata['HouseCountScaled']
# recs.microdata['GasExp_weighted'] = recs.microdata['GasExp'] * recs.microdata['HouseCountScaled']
# recs.microdata['Heated Floor Area_weighted'] = recs.microdata['Heated Floor Area'] * recs.microdata['HouseCountScaled']
# g = recs.microdata[recs.microdata['Heating Fuel'] == 'Natural Gas'].groupby('RD').sum()
# g['gas_avg'] = g['gas_weighted'] / g['HouseCountScaled']
# g['gas_per_sf'] = g['gas_weighted'] / g['Heated Floor Area']
# g['HDD_avg'] = g['HDD_weighted'] / g['HouseCountScaled']
# g['Gas Cost'] = g['GasExp_weighted'] / (g['gas_weighted'] / 100)
# g['HFA_avg'] = g['Heated Floor Area_weighted'] / (g['HouseCountScaled'])

###Explore Aux Heat

In [14]:
# Pool or Tub
recs.microdata.ix[(recs.microdata['Has Heated Pool'] == 1) | (recs.microdata['Has Hot Tub'] == 1), 'Heated Pool or Spa'] = 1

In [15]:
aux_cols = [col for col in recs.microdata.columns if col.startswith('Aux')]
gas_used_cols = [col for col in recs.microdata.columns if 'Gas Used' in col]
fuel_cols = ['gas','electricity']
pool_cols = ['Heated Pool or Spa','Pool Heating Fuel','Hot Tub Fuel']
cols_to_keep = ['HouseCountScaled','CR','RD','Vintage','Heating Fuel','Heated Floor Area','Elec Used Main Heat','Elec Used Aux Heat'] + fuel_cols + gas_used_cols + pool_cols
df = recs.microdata.loc[:,cols_to_keep]
df = df.replace(-2,pd.np.nan)
for col in fuel_cols:
    df.loc[:,col] = df.loc[:,col] * df.loc[:,'HouseCountScaled']

In [16]:
df.loc[:,'samples'] = 1
# groups = ['CR','Vintage','Heating Fuel']
# groups = ['CR','Heating Fuel'] + gas_used_cols + pool_cols
groups = ['CR','Elec Used Main Heat','Elec Used Aux Heat'] + gas_used_cols # for RECS 2005 comparison
# groups = ['CR','Heating Fuel','Gas Used'] + pool_cols
# groups = ['CR','Heating Fuel','Aux Heat Used']
grouped = df.groupby(groups)
sums = grouped.sum().reset_index()
# for col in aux_cols:
#     sums.loc[:,col] = sums[col] / sums['HouseCountScaled']
for col in fuel_cols:
    sums.loc[:,'{} per hh'.format(col)] = sums[col] / sums['HouseCountScaled']
for col in fuel_cols:
    sums.loc[:,'{} per sf'.format(col)] = sums[col] / sums['Heated Floor Area']
sums

,CR,Elec Used Main Heat,Elec Used Aux Heat,Gas Used,Gas Used Main Heat,Gas Used Aux Heat,Gas Used DHW,Gas Used Cooking,Gas Used Other,HouseCountScaled,RD,Heated Floor Area,gas,electricity,Heated Pool or Spa,samples,gas per hh,electricity per hh,gas per sf,electricity per sf
0,CR02,0,0,0,0,0,0,0,0,522495.974303,668,184880,0.000000,22104936.089079,3,68,0.000000,42.306424,0.000000,119.563696
1,CR02,0,0,1,0,0,0,0,1,5756.093733,10,4831,18879.987444,530723.354370,NaN,1,3.280000,92.202000,3.908091,109.857867
2,CR02,0,0,1,0,0,1,0,0,15739.420913,19,3250,382673.439634,499501.414844,NaN,2,24.313057,31.735692,117.745674,153.692743
3,CR02,0,0,1,0,0,1,0,1,11973.026200,9,1842,179176.337083,226481.763599,NaN,1,14.965000,18.916000,97.272713,122.954269
4,CR02,0,0,1,0,0,1,1,0,10776.337900,9,2028,346836.435312,550099.720781,NaN,1,32.185000,51.047000,171.023883,271.252328
5,CR02,0,0,1,1,0,0,0,0,165814.547516,249,60094,11926564.097869,5826702.252680,NaN,26,71.927127,35.139874,198.465140,96.959801
6,CR02,0,0,1,1,0,0,0,1,32286.784123,39,9159,2742777.974906,1184023.068728,2,4,84.950485,36.672066,299.462602,129.274273
7,CR02,0,0,1,1,0,0,1,0,54087.890500,65,13532,3986205.214132,1344549.027504,NaN,7,73.698663,24.858596,294.576206,99.360703
8,CR02,0,0,1,1,0,0,1,1,4193.993085,10,1100,301350.985137,260539.238426,NaN,1,71.853000,62.122000,273.955441,236.853853
9,CR02,0,0,1,1,0,1,0,0,527372.852643,809,216073,50522866.566890,15743012.362231,3,84,95.801038,29.851769,233.823136,72.859693


In [17]:
sums.to_csv('recs_aux.csv')

In [18]:
recs.microdata.to_csv('recs_microdata.csv')

In [20]:
recs_microdata

,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,TOTALBTUWTH_WEIGHTED,TOTALBTURFG_WEIGHTED,TOTALBTUOTH_WEIGHTED,TOTALDOL_WEIGHTED,TOTALDOLSPH_WEIGHTED,TOTALDOLCOL_WEIGHTED,TOTALDOLWTH_WEIGHTED,TOTALDOLRFG_WEIGHTED,TOTALDOLOTH_WEIGHTED,HDD_weighted
0,1,2,4,12,2,2471.679705,4742,1080,4953,1271,...,2.503317e+07,1.278106e+07,6.516089e+07,3250258.812075,561071.293035,541297.855395,521524.417755,266941.408140,1359423.837750,1.172071e+07
1,2,4,10,26,2,8599.172010,2662,199,2688,143,...,2.589211e+08,3.843830e+07,2.549912e+08,11118729.408930,3164495.299680,146185.924170,2467962.366870,1040499.813210,4308185.177010,2.289100e+07
3,4,2,3,7,2,18003.639600,6034,672,5781,868,...,1.159434e+08,1.026207e+08,3.679584e+08,25169088.160800,7651546.830000,1818367.599600,3222651.488400,2862578.696400,9613943.546400,1.086340e+08
5,6,2,4,10,2,4232.486778,8866,270,8708,436,...,6.011824e+07,1.329424e+07,9.297927e+07,8067119.798868,2899253.442930,190461.905010,554455.767918,639105.503478,3779610.692754,3.752523e+07
6,7,1,2,3,2,7862.341967,5213,749,5355,796,...,1.822491e+08,4.305418e+07,3.756863e+08,33839519.825968,9835789.800717,1989172.517651,3074175.709097,2610297.533044,16330084.265459,4.098639e+07
7,8,3,5,17,2,6297.038285,1086,2647,1070,2554,...,9.727665e+07,1.596929e+07,2.119961e+08,16510834.383270,1196437.274150,4414223.837785,3261865.831630,535248.254225,7103059.185480,6.838584e+06
9,10,2,4,12,2,3242.224473,4687,1205,4755,1467,...,3.687058e+07,1.025191e+07,5.470605e+07,4500207.568524,1906427.990124,308011.324935,619264.874343,262620.182313,1403883.196809,1.519631e+07
11,12,1,2,4,2,15489.574010,4732,1034,4668,1437,...,4.590800e+08,1.214073e+08,7.207299e+08,56784778.320660,12329700.911960,3794945.632450,6799922.990390,6490131.510190,27370077.275670,7.329666e+07
15,16,3,5,14,2,37518.828060,4801,946,5222,895,...,2.301405e+08,2.098803e+08,9.199992e+08,79239764.862720,29452280.027100,9154594.046640,6865945.534980,6265644.286020,27463782.139920,1.801279e+08
16,17,3,5,16,2,15622.341370,3423,1604,3400,1641,...,0.000000e+00,1.509118e+07,7.219084e+07,4233654.511270,0.000000,906095.799460,0.000000,578026.630690,2765154.422490,5.347527e+07
